### Imports

In [1]:
import pandas as pd

import requests
import requests.auth
import json

import time

### Get Authorization

In [4]:
# request a token
client_auth = requests.auth.HTTPBasicAuth('ZfnXA82bMZkXWBRp_29tqw', 'MTr-tnMjbizJ9jeuPQI8EK-CiPUFWA')
post_data = {'grant_type': 'password', 'username': 'gtaylor38gadsi', 'password': 'axfig654$Bin'}
headers = {'User-Agent': 'PostScraperScript/1.0 by gtaylor38gadsi'}
response_token = requests.post('https://www.reddit.com/api/v1/access_token', auth=client_auth, data=post_data, headers=headers)
response_token.json()

{'access_token': '2205940059860-Pu6brezMzlKjawNDYJFoqCJnXx8eFg',
 'token_type': 'bearer',
 'expires_in': 86400,
 'scope': '*'}

In [5]:
# store credentials
access_token = response_token.json()['access_token']
token_type = response_token.json()['token_type']

In [6]:
# update headers for collection
headers = {"Authorization": f'{token_type} {access_token}', 'User-Agent': 'PostScraperScript/1.0 by gtaylor38gadsi'}

### Collect Data

In [7]:
# initialize df
posts_df = pd.DataFrame(columns=['title', 'subreddit', 'score', 'author_flair_text', 'link_flair_text', 'created_utc', 'num_comments'])

# set base url
url_base = 'https://oauth.reddit.com/new/.json'

In [8]:
# make initial request, set time, initialize page list, define columns to scrape
response = requests.get(url_base, headers=headers)
curr_time = time.time()
page_new = []
base_cols = ['title', 'subreddit', 'score', 'author_flair_text', 'link_flair_text', 'created_utc', 'num_comments']

# get data from first page using base url and store in posts_df
for i in range(response.json()['data']['dist']):
    row = [response.json()['data']['children'][i]['data'][col_name] for col_name in base_cols]
    page_new.append(row)
df = pd.DataFrame(page_new, columns=posts_df.columns)
df['age'] = curr_time - df['created_utc']
posts_df = pd.concat([posts_df, df])

# collect data from next 39 pages
for i in range(39):
    # update after and url_current
    after = response.json()['data']['after']
    url_current = f'{url_base}?after={after}'

    # make new request, update time, and reset page list
    response = requests.get(url_current, headers=headers)
    curr_time = time.time()
    page_new = []

    # collect and store data in page list
    for j in range(response.json()['data']['dist']):
        row = [response.json()['data']['children'][j]['data'][col_name] for col_name in posts_df.columns if col_name != 'age']
        page_new.append(row)

    # store page list as df, add age column, and concat df to posts_df
    df = pd.DataFrame(page_new, columns=base_cols)
    df['age'] = curr_time - df['created_utc']
    posts_df = pd.concat([posts_df, df])
    
# report number of unique posts
len(posts_df.title.unique())

### Save Data

In [12]:
# Write posts_df to csv:
posts_df.to_csv('data/sports_posts3.csv', index=False)